In [80]:
import os
from PIL import Image
import numpy as np
from numpy.linalg import norm, pinv
np.set_printoptions(precision=12, suppress=True)

In [81]:
consider_distort = False # flag whether to consider distortion
zoomed_coords = True # whether to divide coordinates by 5 or not (zoomed coords or not)

In [82]:
folder_name = 'data/star_coords/spring2016_txt/' # folder with coords files

In [83]:
# 20160218
# fnames = [
#     '20160218-201934-437.txt',
#     '20160218-211934-484.txt'
# ]

# 20160219
# fnames = [
#     '20160219-001934-484.txt',
#     '20160219-021934-593.txt',
#     '20160219-041934-593.txt'
# ]

# 20160311
fnames = [
#     '20160311-020811-125.txt',
    '20160311-030811-125.txt'
]

In [84]:
if consider_distort:
    print "consider_distort is True!"

In [85]:
coords_list = []
for fname in fnames:
    piece = np.loadtxt(folder_name + os.sep + fname)
    coords_list.append(piece)

coords = np.vstack(coords_list)
print 'Zoomed In Star coordinates pairs:\n', coords, '\n'

if zoomed_coords:
    coords /= 5.0
    coords = coords.round()
    print 'Normal Star coordinates pairs:\n', coords

Zoomed In Star coordinates pairs:
[[ 4695.  2972.  5879.  3136.]
 [ 3291.  3114.  4459.  3289.]
 [ 2503.  2618.  3682.  2809.]
 [ 1453.  2075.  2651.  2286.]
 [ 1275.  1099.  2473.  1331.]] 

Normal Star coordinates pairs:
[[  939.   594.  1176.   627.]
 [  658.   623.   892.   658.]
 [  501.   524.   736.   562.]]


In [86]:
lX = coords[:, 0] # leftX coordinates
lY = coords[:, 1] # leftY coordinates
rX = coords[:, 2] # rightX coordinates
rY = coords[:, 3] # rightY coordinates

N = coords.shape[0] # number of pairs of points
M = coords.shape[1] # lX, lY, rX, rY == 4
print 'Number of Star coordinates pairs:', N

Number of Star coordinates pairs: 3


In [87]:
xi = np.zeros(2*N)

num_coeffs = 6
if consider_distort:
    num_coeffs = 8

z = np.zeros(num_coeffs)
arr = np.zeros((2*N, num_coeffs)) # matrix A
print 'Number of coefficients:', num_coeffs

Number of coefficients: 6


In [88]:
for i in xrange(N): # fill the xi vector
    xi[2*i] = rX[i]
    xi[2*i + 1] = rY[i]

In [89]:
if consider_distort:
    im = Image.open('data/stars/2016spring/mod_'+ fnames[0][:-4] +'-1.png')
    x_c = im.width / 2.0
    y_c = im.height / 2.0
    for i in xrange(N): # fill the A matrix
        dist_l = (lX[i]-x_c)**2 + (lY[i]-y_c)**2
        dist_r = (rX[i]-x_c)**2 + (rY[i]-y_c)**2

        zx1 = (lX[i] - x_c) * dist_l
        zx2 = (rX[i] - x_c) * dist_r
        arr[2*i] = [lX[i], lY[i], 0, 0, 1, 0, -zx1, zx2]

        zy1 = (lY[i] - y_c) * dist_l
        zy2 = (rY[i] - y_c) * dist_r
        arr[2*i + 1] = [0, 0, lX[i], lY[i], 0, 1, -zy1, zy2]

else:
    for i in xrange(N): # fill the A matrix
        arr[2*i] = [lX[i], lY[i], 0, 0, 1, 0]

        arr[2*i + 1] = [0, 0, lX[i], lY[i], 0, 1]

In [90]:
np.set_printoptions(precision=2, suppress=True)
print 'A:\n', arr, '\n'
print 'xi:\n', xi
np.set_printoptions(precision=12, suppress=True)

A:
[[ 939.  594.    0.    0.    1.    0.]
 [   0.    0.  939.  594.    0.    1.]
 [ 658.  623.    0.    0.    1.    0.]
 [   0.    0.  658.  623.    0.    1.]
 [ 501.  524.    0.    0.    1.    0.]
 [   0.    0.  501.  524.    0.    1.]] 

xi:
[ 1176.   627.   892.   658.   736.   562.]


In [91]:
p_arr = pinv(arr)
z = np.dot(p_arr, xi)
print 'Affine coefficients: \n', z

Affine coefficients: 
[   1.008278759422   -0.02322995181    -0.008803904609    0.98365871741
  243.024836278301   50.973588286168]


In [92]:
sigma = 2 # 2 pix error
I = np.eye(2*N)

print 'cond(A): ', np.linalg.cond(arr)
proba = (2*N - num_coeffs) / norm( np.dot(I - np.dot(arr, p_arr), xi) )**2
sig_theory = (1 / proba)**0.5

print 'Probability:', proba
print 'Sigma theory:', sig_theory
print 'Sigma practice', sigma

cond(A):  13652.9502215
Probability: 0.0
Sigma theory: inf
Sigma practice 2


/home/stan/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:6: RuntimeWarning: divide by zero encountered in double_scalars


In [93]:
res_fname = 'results/spring2016/'

if consider_distort:
    res_fname += 'affdis_coeffs_shots'
else:
    res_fname += 'affine_coeffs_shots'

res_fname += str(len(fnames)) + '_' + fnames[0]

In [94]:
np.savetxt(res_fname, z, fmt="%.10f")
print 'Coeffs are saved to:', res_fname

Coeffs are saved to: results/spring2016/affine_coeffs_shots1_20160311-030811-125.txt
